<a href="https://colab.research.google.com/github/shrutin567/Earth-Observation/blob/main/land_cover_classification_gee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clustering

Authenticate and Initialize GEE

In [ ]:
import ee

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=g3BqP7gcrlmwcZ8ntJKqD2LNSZXtTpDcZjwGRZoQsCY&tc=6wra607y-p1mMZv47Tvr6jKDCZpaR5xUr8XO9boPC64&cc=SyWjm0XaeQguE5Fw1WGIJ3ptkUg2R87hkF_vP0uIx-Q

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BVz8U5UIQZOkKaXgCnWhPGJxlqrK_dz1nwTh7fAIWWONGg0dbyERlU

Successfully saved authorization token.


In [ ]:
#define region of interest
region=ee.Geometry.Rectangle(31.56,-26.24,31.78,-26.09)

#load landsat data
image=ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
.filterDate('2017-01-01','2017-12-31')\
.filterBounds(region)\
.sort('CLOUD_COVER')\
.first()

#make the training data
training=image.sample(**{
    'region':region,
    'scale':30,
    'numPixels':5000
})

#initiate the clusterer and train it
clusterer=ee.Clusterer.wekaKMeans(15).train(training)

#Cluster the input using the trained clusterer
result=image.cluster(clusterer)

In [ ]:
#import the Folium library
import folium

#define a method for displaying earth engine image tiles to folium map
def add_ee_layer(self,ee_image_object,vis_params,name):
  map_id_dict=ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine>/a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

#Add EE drawing method to folium
folium.Map.add_ee_layer=add_ee_layer

#create a folium map object
my_map=folium.Map(location=[-26.1694, 31.6632], zoom_start=11)

#add the layer to the map object
my_map.add_ee_layer(result.clip(region).randomVisualizer(),{},'clusters')

#add a layer control panel to the map
my_map.add_child(folium.LayerControl())

#display the map
display(my_map)

CART Supervised Classification

In [ ]:
import ee

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=n_VZI1YjGXpGQln8gRQWLsY7x5MGC4vY4qUu3bQXfEU&tc=uW7pCDqYr4R7mixRfaHgEBQpV_3RZ3rO9gIjw6KMAH8&cc=bUsZw2qJ-sY1L6HtEoA3n3F-dVFn80_Cejr_KYwuog8

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BXYYyWFs6NqqPFqHABhX2g6t4VrwkmgUk8mysuBhqFuHmW2trC0kJQ

Successfully saved authorization token.


In [ ]:
#CART Supervised Classification with Landsat

#ROI
roi=ee.Geometry.Point([-122.26, 37.87])

#Load Landsat 8 data
image=ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
.filterDate('2017-01-01','2017-12-31')\
.filterBounds(roi)\
.sort('CLOUD_COVER')\
.first()

visParamsTrue={'bands':['B4','B3','B2'],'min':0, 'max':3000, 'gamma':1.4}


In [ ]:
#Import training data
training=ee.FeatureCollection('users/midekisa/Train_Cover_CA')
label='Class'
bands=['B1','B2','B3','B4','B5','B6','B7']
input=image.select(bands)

#overlay the points on the imagery to get training
trainImage=input.sampleRegions(**{
    'collection':training,
    'properties':[label],
    'scale':30
})
trainingData=trainImage.randomColumn()
trainSet=trainingData.filter(ee.Filter.lessThan('random',0.8))
testSet=trainingData.filter(ee.Filter.greaterThanOrEquals('random',0.8))

In [ ]:
#classification model
classifier=ee.Classifier.smileCart().train(trainSet,label,bands)

#classify the image
classified=input.classify(classifier)
#print(classfied.getInfo())

#define a palette for the classification
landcoverPalette=[
    '#0c2c84', #water (0)
    '#e31a1c', #urban (1)
    '#005a32', #forest (2)
    'FF8000', #crop (3)
    '969696', #barren (4)
]

In [ ]:
#import the folium library
import folium

#define a method for displaying EE image tiles to folium map
def add_ee_layer(self,ee_image_object, vis_params, name):
  map_id_dict=ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

#add EE drawing method to folium
folium.Map.add_ee_layer=add_ee_layer

#create a folium map object
my_map=folium.Map(location=[38,-124], zoom_start=8)

#add the layer to the map object
my_map.add_ee_layer(image, visParamsTrue, "Landsat 2017")
my_map.add_ee_layer(classified, {'palette': landcoverPalette, 'min':0, 'max':4}, 'classification')

#add a layer control panel to the map
my_map.add_child(folium.LayerControl())

#display the map
display(my_map)

Random Forest Supervised Classification

In [ ]:
#random forest supervised classification with landsat
#roi
roi=ee.Geometry.Point([-122.26,37.87])

#load landsat 8 data
image=ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
.filterDate('2017-01-01','2017-12-31')\
.filterBounds(roi)\
.sort('CLOUD_COVER')\
.first()

visParamsTrue={'bands':['B4','B3','B2'], 'min':0, 'max': 3000, 'gamma':1.4}

#import training data
training=ee.FeatureCollection('users/midekisa/Train_Cover_CA')
label='Class'
bands=['B1','B2', 'B3', 'B4', 'B5', 'B6', 'B7' ]
input=image.select(bands)

#overlay the points on the imagery to get training
trainImage=input.sampleRegions(**{
    'collection':training,
    'properties':[label],
    'scale':30
})

trainingData=trainImage.randomColumn()
trainSet=trainingData.filter(ee.Filter.lessThan('random',0.8))
testSet=trainingData.filter(ee.Filter.greaterThanOrEquals('random',0.8))

#classification model
#make a rf classifier and train it
classifier=ee.Classifier.smileRandomForest(10).train(**{
    'features':trainSet,
    'classProperty':label,
    'inputProperties': bands
})

#classify the image
classified=input.classify(classifier);
#print(classified.getInfo());

#define a palette for the classification
landcoverPalette=[
     '#0c2c84', #water (0)
    '#e31a1c', #urban (1)
    '#005a32', #forest (2)
    'FF8000', #crop (3)
    '969696', #barren (4)
]

In [ ]:
#import the folium library
import folium

#define a method for displaying EE image tiles to folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict=ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

#add EE drawing method to folium
folium.Map.add_ee_layer=add_ee_layer

#create a folium map object
my_map=folium.Map(location=[38,-124],zoom_start=8)

#add the layer to the map object
my_map.add_ee_layer(image, visParamsTrue, "Landsat 2017")
my_map.add_ee_layer(classified, {'palette': landcoverPalette, 'min':0, 'max':4}, 'classification')

#add a layer control panel to the map
my_map.add_child(folium.LayerControl())

#display the map
display(my_map)